# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [34]:
#importanto bibliotecas
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [35]:
#df = pd.read_csv('previsao_de_renda.csv')

df = pd.read_csv('/kaggle/input/previsao-de-renda/previsao_de_renda.csv')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   index                  15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12466 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  mau                    15000 non-null  bool   
 15  re

In [37]:
#dropando os dados missing
df = df.dropna()

In [38]:
#dropando colunas indesejadas
df = df.drop(['Unnamed: 0','data_ref','index'], axis = 1)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12466 entries, 0 to 14999
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   12466 non-null  object 
 1   posse_de_veiculo       12466 non-null  bool   
 2   posse_de_imovel        12466 non-null  bool   
 3   qtd_filhos             12466 non-null  int64  
 4   tipo_renda             12466 non-null  object 
 5   educacao               12466 non-null  object 
 6   estado_civil           12466 non-null  object 
 7   tipo_residencia        12466 non-null  object 
 8   idade                  12466 non-null  int64  
 9   tempo_emprego          12466 non-null  float64
 10  qt_pessoas_residencia  12466 non-null  float64
 11  mau                    12466 non-null  bool   
 12  renda                  12466 non-null  float64
dtypes: bool(3), float64(3), int64(2), object(5)
memory usage: 1.1+ MB


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [40]:
#Separando a base em 25% teste e 75% train

df_train, df_test = train_test_split(df,test_size = 0.25, random_state =42) 

In [13]:
#criando o modelo

modelo = ''' renda ~ sexo
                   + posse_de_veiculo
                   + posse_de_imovel
                   + qtd_filhos
                   + tipo_renda
                   + educacao
                   + estado_civil
                   + tipo_residencia
                   + idade
                   + tempo_emprego
                   + qt_pessoas_residencia
                   '''
y_train_OLS, X_train_OLS = patsy.dmatrices(formula_like=modelo, data=df_train)
y_test_OLS, X_test_OLS = patsy.dmatrices(formula_like=modelo, data=df_test)

In [41]:
#estrutura de repetiçao para calcular os valores de r2 para os alphas

X_train_OLS = sm.add_constant(X_train_OLS)
X_test_OLS = sm.add_constant(X_test_OLS)

alphas = [0,0.001,0.005,0.01,0.05,0.1]
r2_y_pred =[]

for alpha in alphas:
    modelo_OLS = sm.OLS(y_train_OLS, X_train_OLS)
    reg = modelo_OLS.fit_regularized(method='elastic_net',
                                     refit=True,
                                     L1_wt=0, #ridge
                                     alpha = alpha)
    y_pred = reg.predict(X_test_OLS)
    r2 = r2_score(y_test_OLS, y_pred)
    r2_y_pred.append(r2)

In [42]:
#criando o dataframe com os alphas e respectivos r2

pd.DataFrame({'alpha':alphas,
              'R2': r2_y_pred})

,alpha,R2
0,0.000,0.207358
1,0.001,0.207839
2,0.005,0.208019
3,0.010,0.208002
4,0.050,0.205750
5,0.100,0.201833


In [43]:
#fazendo a regularizaçao Lasso

r2_y_pred_lasso =[]

for alpha in alphas:
    modelo_OLS = sm.OLS(y_train_OLS, X_train_OLS)
    reg = modelo_OLS.fit_regularized(method='elastic_net',
                                     refit=True,
                                     L1_wt=1, #Lasso
                                     alpha = alpha)
    y_pred = reg.predict(X_test_OLS)
    r2 = r2_score(y_test_OLS, y_pred)
    r2_y_pred_lasso.append(r2)

In [44]:
#criando o dataframe com os alphas e respectivos r2

pd.DataFrame({'alpha':alphas,
              'R2': r2_y_pred_lasso})

,alpha,R2
0,0.000,0.207358
1,0.001,0.207358
2,0.005,0.207358
3,0.010,0.207358
4,0.050,0.207358
5,0.100,0.207358


In [45]:

df_dummies = pd.get_dummies(data=df.dropna(), drop_first=True).astype(int)

X = df_dummies.iloc[:, :-1]
y = df_dummies.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [46]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out=0.05, 
                       verbose=True):
    
    included = list(initial_list)
    while True:
        changed = False
        # forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(dtype=float, index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Adicione  {:30} com p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # null if pvalues is empty
        if worst_pval > threshold_out:
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print('Retire {:30} com p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

resposta = stepwise_selection(X_test, y_test)

Adicione  tipo_residencia_Casa           com p-value 1.36581e-224
Adicione  tipo_residencia_Com os pais    com p-value 5.35263e-288
Adicione  tipo_residencia_Estúdio        com p-value 2.46799e-134
Adicione  tipo_residencia_Comunitário    com p-value 6.76633e-104
Adicione  tempo_emprego                  com p-value 5.03423e-05
Adicione  renda                          com p-value 0.000501871


In [47]:
resposta

['tipo_residencia_Casa',
 'tipo_residencia_Com os pais',
 'tipo_residencia_Estúdio',
 'tipo_residencia_Comunitário',
 'tempo_emprego',
 'renda']

In [48]:
X = df_dummies[resposta]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

y_pred = sm.OLS(y_train, X_train).fit().predict(X_test)
print(f'valor R-squared: {r2_score(y_test, y_pred)}')

valor R-squared: 1.0


O Melhore resultado foi o r2 do modelo stepwise que apresentou 1.0 como valor. Ainda avaliando os modelos o melhor modelo apresentado foi o modelo com o metodo de regularizacao stepwise.

Comparando as regularizaçoes, o modelo com a regularizaçao Ridge teve 2 r2 melhores para 2 alphas que a regularizaçao Lasso que todos os valores para os alphas foram os mesmos. Comparando com o modelo com stepwise a diferença foi muito grande, atingindo um r2 de 1.0. 

Devido ao modelo stepwise ter encontrado um r2 com 1.0, se mostrou o melhor r2 possivel para chegar, nao sendo necessario realizar modificaçoes em variaveis.

In [49]:
# criando a variavel de classificaçao

clf = DecisionTreeClassifier(random_state = 100)

In [50]:
clf = clf.fit(X_test,y_test)

In [ ]:
plt.figure(figsize=(15,20))
plot_tree(clf,
          filled=True,
          rounded=True,
          feature_names=X_test.columns)

In [51]:
predictreg = clf.predict(X_test)

In [52]:
r2_score(y_test,predictreg)

1.0

O R2 obtido pela arvore foi tambem o melhor r2 obtido pelo modelo stepwise. Com um r2 = 1.0